# CS634 Midterm Project
**Student:** Tanushri Vijayakumar  
**Instructor:** Dr. Yasser Abdullah  

## Introduction

This notebook implements and compares three methods for **Association Rule Mining**:  

1. **Brute Force (custom implementation)**  
2. **Apriori (using Python library mlxtend)**  
3. **FP-Growth (using Python library mlxtend)**  

## 1. Setup
Import all required Python libraries:

**os**: Used to handle file paths (locating datasets inside the folder).  
**pandas (pd)**: For working with transaction data in a tabular DataFrame format.  
**time**: To compare runtime
**mlxtend.frequent_patterns**: Contains built-in implementations of: 
- **apriori**: to run the Apriori algorithm.
- **fpgrowth**: to run the FP-Growth algorithm.
- **association_rules**: to generate rules from frequent itemsets

In [1]:
import os
import pandas as pd
import time
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

## 2. Dataset Selection

We are working with transaction datasets representing customer purchases at 5 different stores (Amazon, BestBuy, Walmart, Nike, and Wholefoods.

The user is prompted to:
- Select a dataset
- Specify minimum support and confidence thresholds.

In [3]:
# 1. Transaction database
datasets = {1: "amazon_transactions.csv",2: "bestbuy_transactions.csv", 3: "walmart_transactions.csv",4: "nike_transactions.csv",5: "wholefoods_transactions.csv"}
print("AVAILABLE DATASETS:")

for i, ds in datasets.items():
    print(f"{i}. {ds}")
print("\n")    

data_select  = int(input("Enter the dataset number to choose which dataset to use: "))
if data_select not in datasets:
    print("Invalid selection")
    exit()

filename = os.path.join("..", "dataset", datasets[data_select])

min_sup = float(input("Enter the minimum support threshold (0-1): "))
min_conf = float(input("Enter the minimum confidence threshold (0-1): "))

AVAILABLE DATASETS:
1. amazon_transactions.csv
2. bestbuy_transactions.csv
3. walmart_transactions.csv
4. nike_transactions.csv
5. wholefoods_transactions.csv




Enter the dataset number to choose which dataset to use:  4
Enter the minimum support threshold (0-1):  0.4
Enter the minimum confidence threshold (0-1):  0.5


## 3. Brute Force Implementation

The brute force method generates all possible item combinations and calculates their support by scanning the transactions.  
- **Ck (Candidate Itemsets):** All possible combinations of items of size k.  
- **Lk (Frequent Itemsets):** Subset of Ck that meets the support threshold.  

This process continues until no more frequent itemsets are found.  

Finally, association rules are generated by splitting itemsets into antecedent-consequent pairs and calculating confidence. 

### Steps
1. Load dataset.  
2. Generate all candidate sets Ck.  
3. Prune candidates to get frequent sets Lk.  
4. Repeat for larger k.  
5. Generate rules with sufficient confidence. 

#### Load transactions from dataset

In [4]:
def load_transactions(filename):    
    transactions = []
    with open(filename, "r") as f:
        next(f)  # skip header
        for line in f:
            parts = line.strip().split(",")
            items = [p.strip() for p in parts[3:]]  # items start from 4th column
            transactions.append(items)
    return transactions

#### Generate candidate itemsets of size k

In [5]:
def generate_candidates(items, k):
    candidates = []
    n = len(items)
    items = list(items)

    def helper(start, comb):
        if len(comb) == k:
            candidates.append(comb[:])
            return
        for i in range(start, n):
            comb.append(items[i])
            helper(i+1, comb)
            comb.pop()

    helper(0, [])
    return candidates


#### Calculate support of an itemset

In [6]:
def support_transactions(transactions, item):
    count = 0
    for t in transactions:
        if all(i in t for i in item):
            count += 1
    return count / len(transactions)


#### Generate all frequent itemsets Lk

In [7]:
def frequent_itemsets(transactions, min_sup):
    unique_items = set()
    for t in transactions:
       for i in t:
           unique_items.add(i)

    freq_is = {}
    k = 1

    while True:
        candidates = generate_candidates(unique_items, k)
        valid = []
        for c in candidates:
            sup = support_transactions(transactions, c)
            if sup >= min_sup:
                valid.append((c, sup))
        if not valid:
            break
        freq_is[k] = valid
        k += 1

    return freq_is


#### Generate all non-empty subsets

In [8]:
def generate_subsets(itemset):
    subsets = []

    def helper(start, current):
        if 0 < len(current) < len(itemset):
            subsets.append(current[:])   # store a copy
        for i in range(start, len(itemset)):
            current.append(itemset[i])
            helper(i + 1, current)
            current.pop()

    helper(0, [])
    return subsets


#### Generate association rules

In [9]:
def generate_rules(freq_is, min_conf):
    rules = []
    for k in freq_is:
        for itemset, sup in freq_is[k]:
            if len(itemset) < 2:
                continue

            subsets = generate_subsets(itemset)

            for left in subsets:
                right = [x for x in itemset if x not in left]

                # Find support of left side
                sup_left = None
                for size, sets in freq_is.items():
                    if size == len(left):
                        for c, s in sets:
                            if set(c) == set(left):
                                sup_left = s
                                break

                if sup_left and sup_left > 0:
                    conf = sup / sup_left
                    if conf >= min_conf:
                        rules.append((left, right, sup, conf))
    return rules

#### Calculate time and store in results for comparison at end

In [10]:
import time
results = []

### Brute Force Output

In [11]:
transactions = load_transactions(filename)
start = time.perf_counter()
freq_is = frequent_itemsets(transactions, min_sup)

# Print frequent itemsets or message if none
if not freq_is:
    print("No frequent itemsets found with the given support threshold.")
else:
    for k, items in freq_is.items():
        print("\nFrequent itemsets of size", k, ":")
        for item, sup in items:
            print(f"{item}: {sup:.2f}")

print("\n")
rules = generate_rules(freq_is, min_conf)
end = time.perf_counter()

# Print rules or message if none
if not rules:
    print("No rules generated with the given confidence threshold.")
else:
    print("Association rules:")
    for left, right, sup, conf in rules:
        print(f"{left} -> {right}: (support: {sup:.2f}, confidence: {conf:.2f})")

results.append(["Brute Force", end - start])


Frequent itemsets of size 1 :
['Sweatshirts']: 0.50
['Socks']: 0.55
['"Running Shoe']: 0.70
['Rash Guard']: 0.50

Frequent itemsets of size 2 :
['Sweatshirts', 'Socks']: 0.40
['Sweatshirts', '"Running Shoe']: 0.45
['Socks', '"Running Shoe']: 0.50

Frequent itemsets of size 3 :
['Sweatshirts', 'Socks', '"Running Shoe']: 0.40


Association rules:
['Sweatshirts'] -> ['Socks']: (support: 0.40, confidence: 0.80)
['Socks'] -> ['Sweatshirts']: (support: 0.40, confidence: 0.73)
['Sweatshirts'] -> ['"Running Shoe']: (support: 0.45, confidence: 0.90)
['"Running Shoe'] -> ['Sweatshirts']: (support: 0.45, confidence: 0.64)
['Socks'] -> ['"Running Shoe']: (support: 0.50, confidence: 0.91)
['"Running Shoe'] -> ['Socks']: (support: 0.50, confidence: 0.71)
['Sweatshirts'] -> ['Socks', '"Running Shoe']: (support: 0.40, confidence: 0.80)
['Sweatshirts', 'Socks'] -> ['"Running Shoe']: (support: 0.40, confidence: 1.00)
['Sweatshirts', '"Running Shoe'] -> ['Socks']: (support: 0.40, confidence: 0.89)
['Soc

## 4. Apriori Implementation
The Apriori algorithm is an efficient version of the brute force method. Property: All subsets of a frequent itemset must also be frequent.

This allows the algorithm to prune many candidate sets early, saving time.

### Steps
1. Convert transactions into a one-hot encoded DataFrame.  
2. Use **mlxtend.frequent_patterns.apriori** to extract frequent itemsets.  
4. Use **association_rules** to generate rules based on minimum confidence.

In [12]:
df = pd.DataFrame(False, index=range(len(transactions)), 
                  columns=sorted(set(i for t in transactions for i in t)))
for i, t in enumerate(transactions):
    for item in t:
        df.loc[i, item] = True

start = time.perf_counter()
freq_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
print("\nFrequent Itemsets (Apriori):")
print(freq_itemsets)

rules = association_rules(freq_itemsets, metric="confidence", min_threshold=min_conf)

end = time.perf_counter()
results.append(["Apriori", end - start])

print("\nAssociation Rules (Apriori):")
print(rules[['antecedents','consequents','support','confidence']])



Frequent Itemsets (Apriori):
   support                             itemsets
0     0.70                      ("Running Shoe)
1     0.50                         (Rash Guard)
2     0.55                              (Socks)
3     0.50                        (Sweatshirts)
4     0.50               ("Running Shoe, Socks)
5     0.45         ("Running Shoe, Sweatshirts)
6     0.40                 (Sweatshirts, Socks)
7     0.40  ("Running Shoe, Sweatshirts, Socks)

Association Rules (Apriori):
                     antecedents                   consequents  support  \
0                ("Running Shoe)                       (Socks)     0.50   
1                        (Socks)               ("Running Shoe)     0.50   
2                ("Running Shoe)                 (Sweatshirts)     0.45   
3                  (Sweatshirts)               ("Running Shoe)     0.45   
4                  (Sweatshirts)                       (Socks)     0.40   
5                        (Socks)                 (Sweatshi

## 5. FP-Growth Implementation
The FP-Growth algorithm avoids candidate generation altogether.  
It builds a compact FP-tree structure to represent transactions and mines frequent itemsets directly from the tree.


### Steps
1. Convert transactions into a one-hot encoded DataFrame.  
2. Use **mlxtend.frequent_patterns.fpgrowth** to extract frequent itemsets.  
3. Use **association_rules** to generate rules.  

In [13]:
start = time.perf_counter()

feq_itemsets = fpgrowth(df, min_support=min_sup, use_colnames=True)
print("\nFrequent Itemsets (FP-Growth):")
print(freq_itemsets)

rules = association_rules(freq_itemsets, metric="confidence", min_threshold=min_conf)

end = time.perf_counter()
results.append(["FP-Growth", end - start])

print("\nAssociation Rules (FP-Growth):")
print(rules[['antecedents','consequents','support','confidence']])



Frequent Itemsets (FP-Growth):
   support                             itemsets
0     0.70                      ("Running Shoe)
1     0.50                         (Rash Guard)
2     0.55                              (Socks)
3     0.50                        (Sweatshirts)
4     0.50               ("Running Shoe, Socks)
5     0.45         ("Running Shoe, Sweatshirts)
6     0.40                 (Sweatshirts, Socks)
7     0.40  ("Running Shoe, Sweatshirts, Socks)

Association Rules (FP-Growth):
                     antecedents                   consequents  support  \
0                ("Running Shoe)                       (Socks)     0.50   
1                        (Socks)               ("Running Shoe)     0.50   
2                ("Running Shoe)                 (Sweatshirts)     0.45   
3                  (Sweatshirts)               ("Running Shoe)     0.45   
4                  (Sweatshirts)                       (Socks)     0.40   
5                        (Socks)                 (Swea

## 6. Execution time comparison

To evaluate the efficiency of the three algorithms, we measured the execution time for each: Brute Force, Apriori, FP-Growth

In [14]:
df_times = pd.DataFrame(results, columns=["Algorithm", "Execution Time (seconds)"])
print(df_times)

     Algorithm  Execution Time (seconds)
0  Brute Force                  0.040223
1      Apriori                  0.046616
2    FP-Growth                  0.010354
